# Connection à l'API de Gemini

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
google_API_TOKEN = os.getenv("GOOGLE_API_KEY")

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [ ]:
llm.invoke("Sing a ballad of LangChain.")

In [7]:
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
import datetime
from discord import Poll

@tool
def create_poll(
    question: str,
    answers: str,
    duration: int = 24,
    answers_emoji: str = None,
    multiple: bool =True,
):
    """
    Commande pour créer un sondage avec réactions.
    """
    answers = answers.split()
    answers_emoji = answers_emoji.split()
    
    poll = Poll(question,datetime.timedelta(hours=duration),multiple=multiple)

    for answer, emoji in zip(answers, answers_emoji):
        print(f'Answers :{answer},{emoji}')
        poll.add_answer(text=answer, emoji=emoji)

    return Poll


tools = [create_poll]

graph = create_react_agent(llm, tools=tools)

In [5]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [12]:
inputs = {"messages": [("user", "Je veux que tu crées un sondage simple et attractif sur les fruits. La question principale doit être : 'Quel est votre fruit préféré ?'. Propose des choix de réponse variés et populaires, comme :Pomme 🍏, Banane 🍌, Orange 🍊,Fraise🍓, Mangue 🥭, Raisin 🍇")]}
print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Je veux que tu crées un sondage simple et attractif sur les fruits. La question principale doit être : 'Quel est votre fruit préféré ?'. Propose des choix de réponse variés et populaires, comme :Pomme 🍏, Banane 🍌, Orange 🍊,Fraise🍓, Mangue 🥭, Raisin 🍇
================================== Ai Message ==================================
Tool Calls:
  create_poll (7c565f9f-0024-4df9-9e36-0060ac0d6928)
 Call ID: 7c565f9f-0024-4df9-9e36-0060ac0d6928
  Args:
    answers: Pomme 🍏, Banane 🍌, Orange 🍊, Fraise 🍓, Mangue 🥭, Raisin 🍇
    question: Quel est votre fruit préféré ?
================================= Tool Message =================================
Name: create_poll

Error: AttributeError("'NoneType' object has no attribute 'split'")
 Please fix your mistakes.
================================== Ai Message ==================================

Il y a eu une erreur lors de la création du sondage. Je vais réessayer en

In [13]:
from typing import List

# this is the state schema used by the prebuilt create_react_agent we'll be using below
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain_core.documents import Document


class State(AgentState):
    docs: List[str]

In [14]:
from typing import List, Tuple
from typing_extensions import Annotated

from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langgraph.prebuilt import InjectedState


@tool
def get_context(question: str, state: Annotated[dict, InjectedState]):
    """Get relevant context for answering the question."""
    return "\n\n".join(doc for doc in state["docs"])

In [ ]:
from langgraph.prebuilt import ToolNode, create_react_agent
from langgraph.checkpoint.memory import MemorySaver

tools = [get_context]

# ToolNode will automatically take care of injecting state into tools
tool_node = ToolNode(tools)

checkpointer = MemorySaver()
graph = create_react_agent(llm, tools, state_schema=State, checkpointer=checkpointer)

In [ ]:
docs = [
    "FooBar company just raised 1 Billion dollars!",
    "FooBar company was founded in 2019",
]

inputs = {
    "messages": [{"type": "user", "content": "what's the latest news about FooBar"}],
    "docs": docs,
}
config = {"configurable": {"thread_id": "1"}}
for chunk in graph.stream(inputs, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

In [1]:
from datetime import datetime, timedelta

def prochain_mercredi(date_reference: datetime = None) -> datetime:
    """
    Retourne la date du prochain mercredi à partir de la date de référence donnée.
    Si aucune date de référence n'est fournie, utilise la date actuelle.
    """
    date_reference = date_reference.date() if date_reference else datetime.today().date()
    
    # Calcul des jours à ajouter pour atteindre mercredi (mercredi = 2)
    jours_a_ajouter = (2 - date_reference.weekday()) % 7 or 7  # Assure le mercredi suivant
    
    return datetime.combine(date_reference + timedelta(days=jours_a_ajouter), datetime.min.time())

def prochain_mercredi(date_reference: datetime = None) -> datetime:
    """
    Retourne la date du prochain mercredi à 21h15 à partir de la date de référence donnée.
    Si aucune date de référence n'est fournie, utilise la date actuelle.
    
    :param date_reference: Une date de référence optionnelle (datetime).
    :return: Un objet datetime représentant le prochain mercredi à 21h15.
    """
    date_reference = date_reference if date_reference else datetime.today()
    
    # Calcul des jours à ajouter pour atteindre mercredi (mercredi = 2)
    jours_a_ajouter = (2 - date_reference.weekday()) % 7 or 7  # Assure le mercredi suivant
    
    return datetime.combine(date_reference.date() + timedelta(days=jours_a_ajouter), datetime.min.time().replace(hour=21, minute=15))

def discord_timestamps(date: datetime, format: str = 'f') -> str:
    """
    Génère un timestamp Discord à partir d'une date.
    
    :param date: Un objet datetime
    :param format: Format du timestamp Discord ('F', 'f', 'D', 'd', 'T', 't', 'R')
    :return: Une chaîne de caractères compatible avec Discord
    """
    accepted_formats = {'F', 'f', 'D', 'd', 'T', 't', 'R'}
    
    if format not in accepted_formats:
        raise ValueError(f"Format non pris en charge. Formats acceptés : {', '.join(accepted_formats)}")
    
    timestamp = int(date.timestamp())  # Convertir en timestamp Unix
    return f'<t:{timestamp}:{format}>'

In [2]:
discord_timestamps(prochain_mercredi())

'<t:1749064500:f>'

In [3]:
print(f"## Hey! N'oubliez pas de voter pour le film de la watchparty ! La soirée film aura lieu {discord_timestamps(prochain_mercredi())} ({discord_timestamps(prochain_mercredi(), format='R')})")

## Hey! N'oubliez pas de voter pour le film de la watchparty ! La soirée film aura lieu <t:1749064500:f> (<t:1749064500:R>)
